In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import time
import html5lib
import json
import time
import regex as re
from bs4 import BeautifulSoup
from time import sleep

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
 

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

%matplotlib inline

In [3]:
gow3 = pd.read_csv('GoWposts3')

In [4]:
gow3.head()

,subreddit,name,title,post_paragraph,score,subreddit_type,link_flair_text
0,GodofWar,t3_a3wlrs,God of War = Game of the Year Megathread,God of War won in the following categories of ...,1653,public,NaN
1,GodofWar,t3_cbesey,How we know for sure Fimbulwinter has begun at...,I can feel it in my scrote,174,public,Spoilers
2,GodofWar,t3_cba1g2,I don't get why people act like the original G...,Sure the last GoW probably has the best story ...,496,public,NaN
3,GodofWar,t3_cbhszc,My new Chaurlie tattoo!,https://imgur.com/mqzrlHH\n\nBeen working on a...,32,public,NaN
4,GodofWar,t3_cbl6w5,"I always got the hype for this game, but I nev...","Until now, holy sweet mother of Athena....\n\n...",7,public,NaN


In [5]:
gowm = pd.read_csv('./data/GoWposts')

In [17]:
gow3.shape

(1114, 7)

In [7]:
gowm.shape

(951, 7)

In [18]:
gow3.drop_duplicates(keep=False).shape

(1114, 7)

In [15]:
gow3.shape

(1114, 7)

In [20]:
df.shape

(2065, 7)

In [21]:
df.drop_duplicates(keep=False).shape

(1049, 7)

In [22]:
series = df['name'].duplicated()

In [24]:
df.drop_duplicates(keep=False, inplace = True)

In [28]:
df['name'].duplicated(keep = False).value_counts()

True     966
False     83
Name: name, dtype: int64

In [29]:
df['post_paragraph'].duplicated(keep = False).value_counts()

True     964
False     85
Name: post_paragraph, dtype: int64

In [30]:
gowm['post_paragraph'].duplicated(keep = False).value_counts()

False    945
True       6
Name: post_paragraph, dtype: int64

In [32]:
df = pd.read_csv('./data/master_df_cleaned_vectorized.csv')

/Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
df.isnull().sum();

In [36]:
df['post_paragraph'] = df.post_paragraph.fillna('')

In [42]:
df['post_paragraph'] = df['post_paragraph'].replace(np.nan, "")

In [44]:
df['post_paragraph_clean'] = df['post_paragraph_clean'].replace(np.nan, "")
df['title_clean'] = df['title_clean'].replace(np.nan, "")

In [66]:
df.head()

,subreddit,name,title,post_paragraph,score,subreddit_type,link_flair_text,target,title_clean,post_paragraph_clean,...,post_paragraph_clean_zoras,post_paragraph_clean_zou,post_paragraph_clean_zrrpt,post_paragraph_clean_zrrt,post_paragraph_clean_ztranslation,post_paragraph_clean_zubora,post_paragraph_clean_zues,post_paragraph_clean_zuna,post_paragraph_clean_zyaxif,post_paragraph_clean_zzbpcws
0,GodofWar,t3_a3wlrs,God of War = Game of the Year Megathread,God of War won in the following categories of ...,1653,public,NaN,1,game the year megathread,won in the following category game award game ...,...,0,0,0,0,0,0,0,0,0,0
1,GodofWar,t3_car8er,Possible cameo in the next games... (Spoilers),"In this game we see Jormungandr who, in Norse ...",167,public,NaN,1,possible cameo in the next game spoiler,in this game we see jormungandr who norse myth...,...,0,0,0,0,0,0,0,0,0,0
2,GodofWar,t3_cakipc,Valkyries,"I finally beat my first Valkyrie, I ascended t...",142,public,NaN,1,valkyrie,i finally beat my first valkyrie ascended the ...,...,0,0,0,0,0,0,0,0,0,0
3,GodofWar,t3_canoeb,Best boss from each game,What do you think is the best boss from each g...,31,public,Spoilers,1,best bos from each game,what do you think is the best bos from each ga...,...,0,0,0,0,0,0,0,0,0,0
4,GodofWar,t3_cafsjf,WOMEN OF KRATOS,How many women in here that play GOW? Been pla...,277,public,NaN,1,woman kratos,how many woman in here that play gow been play...,...,0,0,0,0,0,0,0,0,0,0


In [47]:
len(df.columns)

15482

In [90]:
features = list(df.columns)

In [91]:
del_list = ['subreddit', 'target', 'title', 'title_clean', 'post_paragraph', 'post_paragraph_clean', 
            'name', 'score', 'subreddit_type', 'link_flair_text']

In [92]:
features = [i for i in features if i not in del_list]

In [93]:
X = df[features]
y = df['target']

In [94]:
X_trim = X.drop(columns = ['title_kotm', 'post_paragraph_clean_liner',
    'post_paragraph_clean_kratos', 'title_link', 'title_botw', 'post_paragraph_clean_botw', 'title_kondo'])



In [95]:
X_train, X_test, y_train, y_test = train_test_split(X_trim, y, random_state=42, stratify=y)

In [96]:
logreg = LogisticRegression()

In [97]:
logreg.fit(X_train, y_train)

/Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [98]:
cross_val_score(logreg, X_train, y_train, cv=5)

/Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logisti

array([0.91269841, 0.90873016, 0.92063492, 0.90039841, 0.94      ])

In [99]:
logreg.score(X_train, y_train)

1.0

In [100]:
logreg.score(X_test, y_test)

0.918854415274463

In [101]:
weights = logreg.coef_
abs_weights = np.abs(weights)

print(abs_weights)

[[3.71580197e-02 3.03176487e-01 1.31362149e-01 ... 4.37976381e-06
  1.61909285e-03 1.53985916e-05]]


In [102]:
coefficients = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(logreg.coef_))], axis = 1)

In [103]:
coefficients.columns = ['word', 'coefficient']

In [104]:
coefficients['coefficient'] = coefficients['coefficient'].abs()


In [105]:
coefficients.sort_values(by = 'coefficient', ascending = False)


,word,coefficient
1248,title_kokiri,1.621107
9095,post_paragraph_clean_linear,1.453421
7419,post_paragraph_clean_goverment,1.309668
3363,post_paragraph_clean_atop,1.196882
8038,post_paragraph_clean_hyper,1.192907
7183,post_paragraph_clean_gank,1.159907
10337,post_paragraph_clean_ooa,1.150857
954,title_government,1.052784
10067,post_paragraph_clean_nikaido,0.954643
1102,title_hypothetical,0.946588
